Taking all data and creating a dataset

In [2]:
#importing stuff
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import os

In [107]:
#read the data
#format: holder = pd.read_csv("filepathname")
folders = ["window 16","window 32","window 5"]
category = ["distance2","abrupt","ideal","obstacle"]
data = []
for i in folders:
    for j in category:
        filenumber = 0
        filepath = os.path.join(i,j,str(filenumber)+".csv")
        while (os.path.isfile(filepath)):
            data.append(pd.read_csv(filepath)[['linkQuality','neighborLinkQuality','RSSI value', 'AVG RSSI value']])
            filenumber += 1
            filepath = os.path.join(i,j,str(filenumber)+".csv")

test = np.array(data)
print(data)
        

[     linkQuality  neighborLinkQuality  RSSI value  AVG RSSI value
0          1.000                  1.0         -32             -37
1          1.000                  1.0         -39             -37
2          1.000                  1.0         -27             -36
3          1.000                  1.0         -20             -30
4          1.000                  1.0         -22             -27
..           ...                  ...         ...             ...
213        0.329                  0.0         -81             -81
214        0.497                  0.0         -81             -81
215        0.000                  0.0         -81             -81
216        0.000                  0.0         -81             -81
217        0.000                  0.0           0               0

[218 rows x 4 columns],      linkQuality  neighborLinkQuality  RSSI value  AVG RSSI value
0            1.0                1.000         -24            -111
1            1.0                1.000         -23 

C:\Users\eyaph\AppData\Local\Temp\ipykernel_3308\2658426519.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test = np.array(data)


In [92]:
testdata = pd.read_csv(os.path.join("window 16","distance2","1.csv"))[['linkQuality','neighborLinkQuality','RSSI value', 'AVG RSSI value']]
column_indices = {name: i for i, name in enumerate(testdata.columns)}
column_indices

{'linkQuality': 0,
 'neighborLinkQuality': 1,
 'RSSI value': 2,
 'AVG RSSI value': 3}

In [110]:
#modifying the split window function


def split_window(features,input_slice,labels_slice,label_columns,column_indices,input_width,label_width):
    #features is all the data (could be the training, validation or testing)
    #so the below 2 lines dont work since the size/shape stacked dataframes can be different
    
    inputs = tf.convert_to_tensor(features[input_slice])
    labels = tf.convert_to_tensor(features[labels_slice])
    if label_columns is not None:
        labels = tf.stack(
            [labels[ :,column_indices[name]] for name in label_columns],
            axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.

    inputs.set_shape([input_width, None])
    labels.set_shape( (label_width, None))

    return inputs, labels

In [112]:
def makedataset(features,total_window,input_slice,labels_slice,label_columns,column_indices,input_width,label_width,slide = 1,batch=32):
    
    
    for i in features:
        sliding = 0
        i = np.array(i,dtype="float32")
        while(i.shape[0]>(sliding + total_window)):
            print(sliding)
            input, label= split_window(i[sliding:total_window+sliding],input_slice,labels_slice,label_columns,column_indices,input_width,label_width)
            #we need some collection mechanism here
            print(input)
            print(label)
            
            sliding+=1
            

input_width = 5
shift = 1
total_window = input_width+shift
label_width = 1
input_slice = slice(0, input_width)
label_start = total_window - label_width
labels_slice = slice(label_start, None)
label_columns = ["RSSI value"]
makedataset(test,total_window,input_slice,labels_slice,label_columns,column_indices,input_width,label_width)


0
tf.Tensor(
[[  1.   1. -32. -37.]
 [  1.   1. -39. -37.]
 [  1.   1. -27. -36.]
 [  1.   1. -20. -30.]
 [  1.   1. -22. -27.]], shape=(5, 4), dtype=float32)
tf.Tensor([[-34.]], shape=(1, 1), dtype=float32)
1
tf.Tensor(
[[  1.   1. -39. -37.]
 [  1.   1. -27. -36.]
 [  1.   1. -20. -30.]
 [  1.   1. -22. -27.]
 [  1.   1. -34. -28.]], shape=(5, 4), dtype=float32)
tf.Tensor([[-33.]], shape=(1, 1), dtype=float32)
2
tf.Tensor(
[[  1.   1. -27. -36.]
 [  1.   1. -20. -30.]
 [  1.   1. -22. -27.]
 [  1.   1. -34. -28.]
 [  1.   1. -33. -29.]], shape=(5, 4), dtype=float32)
tf.Tensor([[-34.]], shape=(1, 1), dtype=float32)
3
tf.Tensor(
[[  1.   1. -20. -30.]
 [  1.   1. -22. -27.]
 [  1.   1. -34. -28.]
 [  1.   1. -33. -29.]
 [  1.   1. -34. -30.]], shape=(5, 4), dtype=float32)
tf.Tensor([[-38.]], shape=(1, 1), dtype=float32)
4
tf.Tensor(
[[  1.   1. -22. -27.]
 [  1.   1. -34. -28.]
 [  1.   1. -33. -29.]
 [  1.   1. -34. -30.]
 [  1.   1. -38. -32.]], shape=(5, 4), dtype=float32)
tf.Tensor

KeyboardInterrupt: 